In [1]:
install.packages('bigrquery')
library(bigrquery)
library(repr)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘rapidjsonr’


Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar



# Clinical and cohort queries
How old are the patients for each case in a given project?

In [ ]:
project <- "isb-cgc-etl"

case_sql <- "select
  submitter_id case_barcode,
  demo__age_at_index age,
  demo__vital_status vital
from `isb-cgc-bq.TCGA.clinical_gdc_current`
where proj__project_id = 'TCGA-BRCA'
"
case_query <- bq_project_query(project, query=case_sql)
case_df <- as.data.frame(bq_table_download(case_query))
head(case_df)

Waiting for authentication in browser...

Press Esc/Ctrl + C to abort

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=465736758727.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&redirect_uri=http%3A%2F%2Flocalhost%3A1410%2F&response_type=code&state=Lxh8QBfCtU



In [ ]:
case_num <- length(unique(case_df[,2]))
hist(case_df[,2], breaks=case_num/2, main='age at index', xlab='age (years)')

In [ ]:
hist(case_df[case_df[,3] == 'Alive',2], breaks=case_num/2, xlim=c(25,90), col='steelblue1', main='', xlab='age (years)')
hist(case_df[case_df[,3] == 'Dead',2], breaks=case_num/2, add=1, col='orange')
legend('topright', bty='n', legend=c('Alive', 'Dead'), fill=c('steelblue1', 'orange'))

# Gene expressionn
What is the mean RNAseq expression (fpkm-uq) of a specific gene in normal and tumor?

Table: *isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current*

In [ ]:
expr_sql <- "select
  case_barcode,
  aliquot_barcode,
  sample_type_name,
  HTSeq__FPKM_UQ
from `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
where project_short_name = 'TCGA-BRCA'
  and gene_name = 'CCNB1'"

expr_query <- bq_project_query(project, query=expr_sql)
expr_df <- bq_table_download(expr_query)
head(expr_df)

In [ ]:
tumor <- unlist(expr_df[expr_df[,3] == 'Primary Tumor',4])
normal <- unlist(expr_df[expr_df[,3] == 'Solid Tissue Normal',4])
metastatic <- unlist(expr_df[expr_df[,3] == 'Metastatic',4])

boxplot(list(normal, tumor, metastatic), names=c('normal', 'tumor', 'metastatic'))

For larger data sets, instead of downloading these data and computing statistics locally we can save ourselves the compute time and generate summary statistics directly in BigQuery.

In [ ]:
summ_sql <- "select
  sample_type_name,
  avg(HTSeq__FPKM_UQ) avg_fpkm,
  stddev(HTSeq__FPKM_UQ) sdv_fpkm,
  max(HTSeq__FPKM_UQ) max_fpkm,
  min(HTSeq__FPKM_UQ) min_fpkm
from `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
where project_short_name = 'TCGA-BRCA'
and gene_name = 'CCNB1'
group by sample_type_name"

summ_query <- bq_project_query(project, query=summ_sql)
summ_df <- bq_table_download(summ_query)
head(summ_df)

# Genome annotation
We can retrieve Gencode annotation data for specific genes from BigQuery tables:

Table: *isb-cgc-bq.GENCODE_versioned.annotation_gtf_hg38_v22*

In [ ]:
annot_sql <- "select * 
from `isb-cgc-bq.GENCODE_versioned.annotation_gtf_hg38_v22` 
where gene_name = 'CCNB1' 
  and transcript_name = 'CCNB1-001'"
annot_query <- bq_project_query(project, query=annot_sql)
annot_df <- bq_table_download(annot_query)
print(annot_df)

# Mutation data
A common question in biological research is what known mutations exist in a given gene in other data sets? Given the gene annotation data above we can query for all mutations that occur within a given gene in the TCGA-BRCA cases.

Table: *isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current*

In [ ]:
mut_sql <- "select case_barcode, 
    Chromosome, 
    Start_Position, 
    End_Position, 
    Reference_Allele, 
    Tumor_Seq_Allele1, 
    Tumor_Seq_Allele2
from `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current`
where Chromosome = 'chr5'
    and Start_Position > 69167135
    and End_Position < 69177358"

mut_query <- bq_project_query(project, query=mut_sql)
mut_df <- bq_table_download(mut_query)
head(mut_df)

In [ ]:
options(repr.plot.width=10, repr.plot.height=4)
mut_count <- table(mut_df[,3])
max_count <- max(mut_count)
plot(NA, xlim=c(min(mut_df[,3])-1000, max(mut_df[,3])+1000), ylim=c(-0.3,max_count), 
     yaxt='n', bty='n', main='', xlab='Chr5 position', ylab='occurance count')
jnk <- apply(annot_df[annot_df$feature == 'exon',], 1, function(row){
    rect(row[4], -0.2, row[5], -0.1, col='green')
})
axis(2, at=seq(0,3,1))
jnk <- sapply(1:length(mut_count), function(ix){
    x <- as.numeric(names(mut_count)[ix])
    segments(x, 0, x, mut_count[ix])
    })

What about the regulatory regions of the gene? What are the mutations in the 100 kB upstream and downstream of the gene?

In [ ]:
sql <- "select case_barcode, 
    Chromosome, 
    Start_Position, 
    End_Position, 
    Reference_Allele, 
    Tumor_Seq_Allele1, 
    Tumor_Seq_Allele2
from `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current`
where 
    Chromosome = 'chr9'
    and (
        (Start_Position < 69067135 and Start_Position > 69167135)
        or (End_Position > 69177358 and End_Position < 69277358)
        )"

query <- bq_project_query(project, query=sql)
df <- bq_table_download(query)
head(df)

# Table joins
Having a given set of gene expression, clinical data, or mutations, how do we join these tables together for more complex analyses such as differential expression analyses or machine learning?

Tables: 
*   *isb-cgc-bq.TCGA.clinical_gdc_current*
*   *isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current*

In [ ]:
sql <- "select 
    mut.Chromosome, 
    mut.Start_Position, 
    mut.End_Position, 
    mut.Reference_Allele, 
    mut.Tumor_Seq_Allele1, 
    mut.Tumor_Seq_Allele2, 
    clin.demo__vital_status 
from `isb-cgc-bq.TCGA.clinical_gdc_current` clin
join `isb-cgc-bq.TCGA.somatic_mutation_hg38_gdc_current` mut 
    on clin.submitter_id = mut.case_barcode
where mut.project_short_name = 'TCGA-BRCA'"

query <- bq_project_query(project, query=sql)
df <- bq_table_download(query)
head(df)